언어모델링
==
- 어린왕자 소설 데이터 https://goo.gl/18hx19
- 음절 단위 언어 모델링

In [8]:
from __future__ import print_function
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

INPUT_FILE = "./data/The_little_prince.txt"

print("Extracting text from input...")
fin = open(INPUT_FILE, 'rb')
lines = []
for line in fin:
    line = line.strip().lower()
    line = line.decode("euckr", "ignore")
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)

# char indexing
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

Extracting text from input...


In [9]:
print(index2char[2])
print(char2index["사"])

빛
661


In [30]:
for i in range(840,860):
    print(index2char[i])

컨
낡
쁜
순
닮
왠
희
죽
뿌
싣


KeyError: 850

In [33]:
text[:100]

'여섯 살 적에 나는 "체험한 이야기"라는 제목의, 원시림에 관한 책에서 기막힌 그림 하나를 본 적이 있다. 맹수를 집어삼키고 있는 보아 구렁이 그림이었다. 위의 그림은 그것을 옮겨'

In [10]:
# X, Y 데이터 생성
# "The sky was falling"
# 아래와 같이 10개 문자를 주면 다음 문자가 정답
#   The sky wa -> s
#   he sky was ->  
#   e sky was  -> f
#    sky was f -> a
#   sky was fa -> l

print("Creating input and label text...")
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

Creating input and label text...


In [11]:
nb_chars

850

In [12]:
# 데이터 벡터화
# 원핫 인코딩 - True False
# 주어진 어린왕자 데이터에는 총 850개의 글자 포함
# input_chars(44792)에서 각 row마다 10개의 글자를 벡터화(850) => (44792, 10, 850)
# 전체 글자에 대해 해당 되는 글자에만 True, 나머지는 False
#
print("Vectorizing input and label text...")
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1
X.shape

Vectorizing input and label text...


(44792, 10, 850)

In [19]:
# np.argmax(X[:3])
np.argmax(X[0,0,:500])

442

In [13]:
# RNN 모델
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 10

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [7]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # 각 학습마다 테스트 데이터( 연속된 10개 글자 )를 부여하고
    # 다음 문자 10개를 예측
    
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("입력: %s" % (test_chars))
    print(test_chars, end="")
    
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
    print()

Iteration #: 0
Epoch 1/1
44792/44792 [==============================] - 16s 348us/step - loss: 4.26210s - loss: 4.2
입력:  물었다. "그럼 
 물었다. "그럼  이 다. " 그 
Iteration #: 1
Epoch 1/1
44792/44792 [==============================] - 17s 382us/step - loss: 3.6831
입력: ..." (위험이 
..." (위험이 그 이 있는 그 이
Iteration #: 2
Epoch 1/1
44792/44792 [==============================] - 14s 313us/step - loss: 3.4193
입력:  연록색 방 속에 
 연록색 방 속에 있는 그 이 있는 
Iteration #: 3
Epoch 1/1
44792/44792 [==============================] - 14s 307us/step - loss: 3.2299
입력:  작은 것이다지구에
 작은 것이다지구에 어린 왕자가 말했
Iteration #: 4
Epoch 1/1
44792/44792 [==============================] - 14s 308us/step - loss: 3.0766
입력:  비밀은 이런 거야
 비밀은 이런 거야. 그래서 그 꽃은
Iteration #: 5
Epoch 1/1
44792/44792 [==============================] - 14s 311us/step - loss: 2.9508
입력:  찾아야 해." 나
 찾아야 해." 나는 그 이 없었다.
Iteration #: 6
Epoch 1/1
44792/44792 [==============================] - 14s 316us/step - loss: 2.8395
입력: 을 계속했다. "그
을 계속했다. "그래서 그 꽃은 그 
Iteration #: 7
Epoch